Dependencies

In [1]:
from openpyxl import workbook,load_workbook
import openpyxl
from openpyxl.styles import PatternFill

import pandas as pd
import numpy as np
import math

Finding Splitter's Availability & Related Details

In [15]:
def LocatingSplitterAndRetrievingAllocatedPorts(splitter_id, filePath, sheetName):
    returningArray = []
    masterIndex = []
    for eachSplitterSheet in sheetName:
        #Loading Sheet
        try:
            DataFrame = pd.read_excel(filePath, sheet_name = eachSplitterSheet)
        except:
            print("No Sheet Found Named ", "/", eachSplitterSheet, "/")
            continue

        # DataFrame to Numpy Array
        NdArray = DataFrame.values
        #Splitter = NdArray[0][5]
        #Port = NdArray[0][6]
        #Range = NdArray[5][12]
        #Master Series = NdArray[5][14]
        #Vendor = NdArray[0][17]

        ReturningData = []
        ReturningData.append([splitter_id])
        ReturningData.append([eachSplitterSheet])
        
        portFound = []
        rangeFound = []
        masterSeriesFound = []
        vendorFound = []

        matchFoundOrNot = 0

        for numberOfRows in range(len(NdArray)):
            if NdArray[numberOfRows][5] == str(splitter_id):
                matchFoundOrNot += 1

                portFound.append(NdArray[numberOfRows][6])
                vendorFound.append(NdArray[numberOfRows][17])

                masterSeriesForSingleSplitterAllocation = []
                index = []
                masterSeriesForSingleSplitterAllocation.append(NdArray[numberOfRows][14])
                index.append(numberOfRows)

                for nextRow in range((numberOfRows + 1), 100):
                    try:
                        if type(NdArray[nextRow][5]) == float:
                            masterSeriesForSingleSplitterAllocation.append(NdArray[nextRow][14])
                            index.append(nextRow)
                        elif NdArray[nextRow][6] == ((NdArray[numberOfRows][6]) + 1):
                            rangeFound.append(NdArray[(nextRow - 1)][12])
                            break
                    except:
                        #For crossing the number or rows available
                        rangeFound.append(NdArray[(nextRow - 1)][12])
                        break
                masterSeriesFound.append(masterSeriesForSingleSplitterAllocation)
                masterIndex.append(index)
            else:
                pass

        ReturningData.append(portFound)
        ReturningData.append(rangeFound)
        ReturningData.append(masterSeriesFound)
        ReturningData.append(vendorFound)

        if len(ReturningData[1]) != 0:
            returningArray = ReturningData
        else:
            pass
        if matchFoundOrNot > 0:
            break
        else:
            pass
        
    return returningArray, masterIndex

Updating Excel Sheet Accordingly

In [28]:
def ReflectingChangesToSplitterSheet(SpliterDetails, Index, UnitsFromLoopQual, availablePortDataFromForte, file_path):

    # Highlighting Function
    def HighlightCellWhereUpdatesMade (changesMadeLogInput):
        try:
            wb = openpyxl.load_workbook(file_path)
            ws = wb[str(SpliterDetails[1][0])]

            processedCellHeaderWithRawInput = []
            for i in range(len(changesMadeLogInput)):
                processedCellHeaderWithRawInput.append(str( "M" + str(changesMadeLogInput[i] + 2)))
                processedCellHeaderWithRawInput.append(str( "N" + str(changesMadeLogInput[i] + 2)))
                processedCellHeaderWithRawInput.append(str( "O" + str(changesMadeLogInput[i] + 2)))
                processedCellHeaderWithRawInput.append(str( "P" + str(changesMadeLogInput[i] + 2)))
                processedCellHeaderWithRawInput.append(str( "Q" + str(changesMadeLogInput[i] + 2)))
                processedCellHeaderWithRawInput.append(str( "R" + str(changesMadeLogInput[i] + 2)))

            fill_cell = PatternFill(patternType='solid', fgColor='FFFF00') #You can give the hex code for different color

            for i in range(len(processedCellHeaderWithRawInput)):
                ws[str(processedCellHeaderWithRawInput[i])].fill = fill_cell

            wb.save(file_path)
        except:
            print("Error Occured While Highlighting")

    # Save
    def ReflectChangesToExcel (filePath, sheetname, dataframe):
        try:
            with pd.ExcelWriter(filePath, engine='openpyxl', mode='a') as writer: 
                workBook = writer.book
                try:
                    workBook.remove(workBook[sheetname])
                except:
                    print("Worksheet does not exist")
                finally:
                    dataframe.to_excel(writer, sheet_name=sheetname,index=False)
                    writer.save()
        except:
            print("Error Occured While Saving")

    # Continue The Sequence
    def continueMasterSeriesSequence():
        seriesNumberMemory = 0 #1001
        localBreakCondition = 0

        try:
            for eachRow in range(len(NdArray)):
                seriesNumberMemory = NdArray[eachRow][14]

                if (int(seriesNumberMemory + 1)) != (int(NdArray[eachRow + 1][14])):
                    for nonSequenceRow in range(eachRow, len(NdArray)):
                        NdArray[nonSequenceRow][14] = seriesNumberMemory
                        seriesNumberMemory += 1
                    localBreakCondition += 1
                else:
                    pass

                if localBreakCondition == 1:
                    break
                else:
                    pass
        except:
            pass

    unitsRequiedForCustomer = UnitsFromLoopQual
    #loading excel to DataFrame
    DataFrame = pd.read_excel(str(file_path), sheet_name = str(SpliterDetails[1][0]))       #------------------------------------------ DataFrame

    # DF to NdArray
    NdArray = DataFrame.values                                                              #------------------------------------------ Numpy Array
    #Delete     arr = np.delete(arr, 0, 0)
    #                          (obj, index, axis 0-row 1-column)
    #Insert     arr = np.insert(arr, [0], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1001, 'POTS', 'ES', 1], axis = 0)
    # SpliterDetails = [['SP3'], ['Sheet2'], [60.0, 61.0], [6, 6], [[1000, 1001, 1002, 1002, 1004, 1005], [1006, 1007, 1008, 1009, 1010, 1011]], ['TA324', 'TA324']]
    # Index = [[0, 1, 2, 3, 4, 5], [6, 7, 8, 9, 10, 11]]
    #-------------[60, 61]


    # Finding Vendor
    VendorName = ""
    if SpliterDetails[5][0][0] == "T" :
        VendorName = "Atron"
    else:
        VendorName = "Calix"


    # 1 UNIT
    breakCondition = 0
    if unitsRequiedForCustomer == 1:
        if len(availablePortDataFromForte) > 0:
            for numberOfPortsFound in range(len(SpliterDetails[2])):
                for numberOfPortsAvailable in range(len(availablePortDataFromForte)):

                    if int(SpliterDetails[2][numberOfPortsFound]) == (availablePortDataFromForte[numberOfPortsAvailable]):
                        loadingIndexForLoopUse = []
                        loadingIndexForLoopUse.append(Index[numberOfPortsFound])

                        while len(loadingIndexForLoopUse[0]) != 2:
                            NdArray = np.delete(NdArray, loadingIndexForLoopUse[0][len(loadingIndexForLoopUse[0])-1], 0)
                            #del loadingIndexForLoopUse[0][len(loadingIndexForLoopUse)-1]
                            loadingIndexForLoopUse[0].remove(int(loadingIndexForLoopUse[0][len(loadingIndexForLoopUse[0])-1]))

                        NdArray[int(loadingIndexForLoopUse[0][len(loadingIndexForLoopUse[0])-1])][15] = "DATA"
                        NdArray[int(loadingIndexForLoopUse[0][len(loadingIndexForLoopUse[0])-1])][16] = "DOE"
                        breakCondition += 1

                        continueMasterSeriesSequence() #To continue the sequence

                        ndarray_to_DataFrame = pd.DataFrame(data=NdArray, index=None, columns=(DataFrame.columns), dtype=None, copy=None)
                        ReflectChangesToExcel (file_path, str(SpliterDetails[1][0]), ndarray_to_DataFrame)

                        changesMadeLog = []
                        for loadingEachIndex in range(len(loadingIndexForLoopUse[0])):
                            changesMadeLog.append(loadingIndexForLoopUse[0][loadingEachIndex])

                        HighlightCellWhereUpdatesMade(changesMadeLog)
                        print("Customer's Unit Order Placed")

                    else:
                        pass
                    if breakCondition == 1:
                        break
                    else:
                        pass
                if breakCondition == 1:
                    break
                else:
                    pass
            unitsRequiedForCustomer -= 1
        else:
            print("Insufficient Available Units")


    # 4 UNIT
    elif unitsRequiedForCustomer == 4:
        if len(availablePortDataFromForte) > 0:
            for numberOfPortsFound in range(len(SpliterDetails[2])):
                for numberOfPortsAvailable in range(len(availablePortDataFromForte)):
                    
                    if int(SpliterDetails[2][numberOfPortsFound]) == int(availablePortDataFromForte[numberOfPortsAvailable]):
                        loadingIndexForLoopUse = []
                        loadingIndexForLoopUse.append(Index[numberOfPortsFound])

                #Condition where number of rows was exactly = 6
                #Unchanged Rows
                        if len(loadingIndexForLoopUse[0]) == 6:
                            #Value Update
                            for sixRows in range(6):
                                if sixRows < 2:
                                    NdArray[loadingIndexForLoopUse[0][sixRows]][15] = "POTS"
                                    NdArray[loadingIndexForLoopUse[0][sixRows]][16] = "ES" 
                                else:
                                    NdArray[loadingIndexForLoopUse[0][sixRows]][15] = "DATA"
                                    NdArray[loadingIndexForLoopUse[0][sixRows]][16] = "DOE"

                #Condition where number of rows was greater than 6
                #Remove Excess Rows
                        elif len(loadingIndexForLoopUse[0]) > 6:
                            #Removing Remaining Rows
                            while len(loadingIndexForLoopUse[0]) != 6:
                                NdArray = np.delete(NdArray, loadingIndexForLoopUse[0][len(loadingIndexForLoopUse[0])-1], 0)
                                loadingIndexForLoopUse[0].remove(int(loadingIndexForLoopUse[0][len(loadingIndexForLoopUse[0])-1]))
                                if len(loadingIndexForLoopUse[0]) == 6:
                                    break
                            #Value Update
                            for sixRows in range(6):
                                if sixRows < 2:
                                    NdArray[loadingIndexForLoopUse[0][sixRows]][15] = "POTS"
                                    NdArray[loadingIndexForLoopUse[0][sixRows]][16] = "ES" 
                                else:
                                    NdArray[loadingIndexForLoopUse[0][sixRows]][15] = "DATA"
                                    NdArray[loadingIndexForLoopUse[0][sixRows]][16] = "DOE"

                #Condition where number of rows was lesser than 6
                #Add required Rows
                        elif len(loadingIndexForLoopUse[0]) < 6:
                            while len(loadingIndexForLoopUse[0]) != 6:
                                demoStructureForRow = NdArray[loadingIndexForLoopUse[0][1]]
                                NdArray = np.insert(NdArray, [(loadingIndexForLoopUse[0][len(loadingIndexForLoopUse[0])-1])+1], demoStructureForRow, axis = 0)
                                NdArray[(loadingIndexForLoopUse[0][len(loadingIndexForLoopUse[0])-1])][12] = int((NdArray[(loadingIndexForLoopUse[0][len(loadingIndexForLoopUse[0])-2])][12]) + 1) #Range Sequence Update
                                loadingIndexForLoopUse[0].append((loadingIndexForLoopUse[0][len(loadingIndexForLoopUse[0])-1])+1)
                                if len(loadingIndexForLoopUse[0]) == 6:
                                    break
                            NdArray[(loadingIndexForLoopUse[0][len(loadingIndexForLoopUse[0])-1])][12] = int(6)
                        
                        breakCondition += 1
                        #Value Update
                        for sixRows in range(6):
                            if sixRows < 2:
                                NdArray[loadingIndexForLoopUse[0][sixRows]][15] = "POTS"
                                NdArray[loadingIndexForLoopUse[0][sixRows]][16] = "ES" 
                            else:
                                NdArray[loadingIndexForLoopUse[0][sixRows]][15] = "DATA"
                                NdArray[loadingIndexForLoopUse[0][sixRows]][16] = "DOE"
                                    
            #General Process to Reflect changes
                        changesMadeLog = []
                        for loadingEachIndex in range(len(loadingIndexForLoopUse[0])):
                            changesMadeLog.append(loadingIndexForLoopUse[0][loadingEachIndex])

                        continueMasterSeriesSequence()

                        ndarray_to_DataFrame = pd.DataFrame(data=NdArray, index=None, columns=(DataFrame.columns), dtype=None, copy=None)
                        ReflectChangesToExcel (file_path, str(SpliterDetails[1][0]), ndarray_to_DataFrame)

                        HighlightCellWhereUpdatesMade(changesMadeLog)
                        print("Customer's Unit Order Placed")
                    else:
                        pass
                    if breakCondition == 1:
                        break
                    else:
                        pass
                if breakCondition == 1:
                    break
                else:
                    pass
            unitsRequiedForCustomer -= 4
        else:
            pass


    # 8 UNIT
    elif unitsRequiedForCustomer == 8:

        if VendorName == "Atron":
            if len(availablePortDataFromForte) >= 2:
                changesMadeLog = []
                for onlyTwoPortIndex in range(len(availablePortDataFromForte)):
                    if onlyTwoPortIndex < 2:
                        for numberOfPortsAvailable in range(len(SpliterDetails[2])):
                            if int(availablePortDataFromForte[onlyTwoPortIndex]) == int(SpliterDetails[2][numberOfPortsAvailable]):
                                for eachIndexOfRowsAvailableInPort in range(len(Index[numberOfPortsAvailable])):
                                    changesMadeLog.append(Index[numberOfPortsAvailable][eachIndexOfRowsAvailableInPort])
                            else:
                                pass
                    elif onlyTwoPortIndex >= 2:
                        break
                continueMasterSeriesSequence()

                ndarray_to_DataFrame = pd.DataFrame(data=NdArray, index=None, columns=(DataFrame.columns), dtype=None, copy=None)
                ReflectChangesToExcel (file_path, str(SpliterDetails[1][0]), ndarray_to_DataFrame)

                HighlightCellWhereUpdatesMade(changesMadeLog)
                print("Customer's Unit Order Placed")
            else:
                print("Insufficient Available Units")
                pass

        elif VendorName == "Calix":
            if len(availablePortDataFromForte) > 0:
                for numberOfPortsFound in range(len(SpliterDetails[2])):
                    for numberOfPortsAvailable in range(len(availablePortDataFromForte)):

                        if int(SpliterDetails[2][numberOfPortsFound]) == int(availablePortDataFromForte[numberOfPortsAvailable]):
                            loadingIndexForLoopUse = []
                            loadingIndexForLoopUse.append(Index[numberOfPortsFound])
                            while len(loadingIndexForLoopUse[0]) != 21:
                                demoStructureForRow = NdArray[loadingIndexForLoopUse[0][1]]
                                NdArray = np.insert(NdArray, [(loadingIndexForLoopUse[0][len(loadingIndexForLoopUse[0])-1])+1], demoStructureForRow, axis = 0)
                                NdArray[(loadingIndexForLoopUse[0][len(loadingIndexForLoopUse[0])-1])][12] = int((NdArray[(loadingIndexForLoopUse[0][len(loadingIndexForLoopUse[0])-2])][12]) + 1)
                                loadingIndexForLoopUse[0].append((loadingIndexForLoopUse[0][len(loadingIndexForLoopUse[0])-1])+1)
                            NdArray[(loadingIndexForLoopUse[0][len(loadingIndexForLoopUse[0])-1])][12] = int(21)
                            breakCondition += 1

                            for indexOfRowsPerUpdatedPort in range(len(loadingIndexForLoopUse[0])):
                                if indexOfRowsPerUpdatedPort <= 7:                                          # 8 POTS
                                    NdArray[loadingIndexForLoopUse[0][indexOfRowsPerUpdatedPort]] [15] = "POTS"
                                    NdArray[loadingIndexForLoopUse[0][indexOfRowsPerUpdatedPort]] [16] = "ES"
                                elif indexOfRowsPerUpdatedPort <= 15 and indexOfRowsPerUpdatedPort > 7:     # 8 DATA
                                    NdArray[loadingIndexForLoopUse[0][indexOfRowsPerUpdatedPort]] [15] = "DATA"
                                    NdArray[loadingIndexForLoopUse[0][indexOfRowsPerUpdatedPort]] [16] = "DOE"
                                elif indexOfRowsPerUpdatedPort <= 20 and indexOfRowsPerUpdatedPort > 15:    # 5 VIDEO
                                    NdArray[loadingIndexForLoopUse[0][indexOfRowsPerUpdatedPort]] [15] = "VIDEO"
                                    NdArray[loadingIndexForLoopUse[0][indexOfRowsPerUpdatedPort]] [16] = "UNKNOWN"

                            changesMadeLog = []
                            for loadingEachIndex in range(len(loadingIndexForLoopUse[0])):
                                changesMadeLog.append(loadingIndexForLoopUse[0][loadingEachIndex])

                            continueMasterSeriesSequence()

                            ndarray_to_DataFrame = pd.DataFrame(data=NdArray, index=None, columns=(DataFrame.columns), dtype=None, copy=None)
                            ReflectChangesToExcel (file_path, str(SpliterDetails[1][0]), ndarray_to_DataFrame)

                            HighlightCellWhereUpdatesMade(changesMadeLog)
                            print("Customer's Unit Order Placed")
                        else:
                            pass
                        if breakCondition == 1:
                            break
                        else:
                            pass
                    if breakCondition == 1:
                        break
                    else:
                        pass
                unitsRequiedForCustomer -= 8
                
            else:
                print("Insufficient Available Units")

    else:
        print("Incorrect Unit Details")

Integrating Function

In [29]:
def SPLITTER_SHEET_MODULE(splitter_id, file_path, sheet_name, unitsRequiedForCustomer, availablePortDataFromForte):
    SpliterDetails, ArrayIndexOfPortLocated = LocatingSplitterAndRetrievingAllocatedPorts(splitter_id, file_path, sheet_name)
    # Output Format : [[Splitter Id], [Sheet], [Port], [Range], [Master Series], [Vendor Model]] 
    print(SpliterDetails)

    ReflectingChangesToSplitterSheet(SpliterDetails, ArrayIndexOfPortLocated, unitsRequiedForCustomer, availablePortDataFromForte, file_path)

Loading Parameters

In [30]:
splitter_id = "SP2"
file_path = "C:/Users/kevin/Desktop/Prodapt Workspace/Excel Feasibility Check/Splitter Sheet/Simulation - Copy.xlsx"
sheet_name = ["Sheet1", "Sheet2"]
unitsRequiedForCustomer = 4
availablePortDataFromForte = [62, 63]

SPLITTER_SHEET_MODULE(splitter_id, file_path, sheet_name, unitsRequiedForCustomer, availablePortDataFromForte)

[['SP2'], ['Sheet1'], [62.0, 63.0], [6, 6], [[1012, 1013, 1014, 1015, 1016, 1017], [1018, 1019, 1020, 1021, 1022, 1023]], ['762GX', '762GX']]
Customer's Unit Order Placed
